In [ ]:
import pickle
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import itertools

mc = False

name = 'add_8c4'#'mc_915'#mc_cc0'#add_cc1'#'mc_7ac'#'add_a2b'
file = f'{name}/stepByStep.pkl'


def get_dim(objectList, key):
    dim = set()
    for obj in objectList:
        assert isinstance(obj, dict)
        assert key in obj.keys()
        if obj[key] is not None and isinstance(obj[key], np.ndarray):
            dim.add(obj[key].shape)
    return dim


with open(file, 'rb') as f:
    print(f"Loading {file}")

    read_heads = 1
    x = pickle.load(f)

    print("keys: ", x.keys())
    if "MEMORYCOLUMNS" in x.keys():
        MEMORYCOLUMNS = x["MEMORYCOLUMNS"]
    elif mc:
        MEMORYCOLUMNS = 16
    elif not mc:
        MEMORYCOLUMNS = 32
    if "INPUTMAXSIZE" in x.keys():
        INPUTMAXSIZE = x["INPUTMAXSIZE"]
    elif mc:
        INPUTMAXSIZE = 1
    elif not mc:
        INPUTMAXSIZE = 192#167#66#16
    if "OUTPUTSIZE" in x.keys():
        OUTPUTSIZE = x["OUTPUTSIZE"]
    elif mc:
        OUTPUTSIZE = 64
    elif not mc:
        OUTPUTSIZE = 64

    OUTPUTSIZE = OUTPUTSIZE*3
    DOUTPUTSIZE = OUTPUTSIZE + read_heads#+ 1

    print(OUTPUTSIZE, DOUTPUTSIZE)

    np.set_printoptions(precision=2, suppress=True, threshold=1000)
    
    #print(x)
    print(x.keys())
    #print(x["total_o"])
    #print(len(x["objects"]))
    x["objects"] = list(filter(lambda obj: obj is not None,  x["objects"]))
    #print(x["objects"])
    #print(x["objects"][0].keys())

    dims = {}
    for key in x["objects"][0].keys():
        dims[key] = get_dim(x["objects"], key)

    for key in dims.keys():
        if not isinstance(dims[key], set):
            continue
        if len(dims[key]) == 0:
            dims[key] = None
            continue
        first = True
        newdim = None
        for el in dims[key]:
            if first:
                newdim = list(el)
                first = False
            else:
                for i,d in enumerate(el):
                    if newdim[i] < d:
                        newdim[i] = d
                        print(f"NEW DIM ({key}): {newdim}")
        dims[key] = el
        
        
            
    
    print("DIMLIST: ", dims)

    if "loss" in x.keys():
        print(f'MSE: {x["loss"]}')

    if "accuracy" in x.keys():
        print(f'Accuracy: {x["accuracy"]}')

    print(x["input"], x["output"], x["target"])

    #raise Exception("STOP")
    for i in range(x["input"].shape[0]):
        fig = px.imshow(x["input"][i,:,:], 
                        text_auto=True,
                        labels=dict(x="Input", y="Time", color="Value"),
                    y=[str(t) for t in range(x["input"].shape[1])],
                    x=[f"input_{j}" for j in range(x["input"].shape[2])])
        fig.write_image(f"{name}/input_{i}.png", width=1920, height=1080)
    
    for i in range(x["output"].shape[0]):
        fig = px.imshow(np.round(x["output"][i,:,:], decimals=2), 
                        text_auto=True,
                        labels=dict(x="Output", y="Time", color="Value"),
                    y=[str(t) for t in range(x["output"].shape[1])],
                    x=[f"output_{j}" for j in range(x["output"].shape[2])])
        fig.write_image(f"{name}/output_{i}.png", width=1920, height=1080)

    for i in range(x["target"].shape[0]):
        fig = px.imshow(x["target"][i,:,:], 
                        text_auto=True,
                        labels=dict(x="Target", y="Time", color="Value"),
                    y=[str(t) for t in range(x["target"].shape[1])],
                    x=[f"target_{j}" for j in range(x["target"].shape[2])])
        fig.write_image(f"{name}/target_{i}.png", width=1920, height=1080)
    
    

   

    maxi = 0
    maxtime = 0
    maxlayer = 0
    for obj in x["objects"]:
        if obj["i"]> maxi:
            maxi = obj["i"]
        if obj["time"] > maxtime:
            maxtime = obj["time"]
        if obj["layer"] > maxlayer:
            maxlayer = obj["layer"]
    #print(maxi, maxtime, maxlayer)

    d = {}
    keys = set()

    for obj in x["objects"]:
        for k in obj.keys():
            if isinstance(obj[k], dict):
                for k2 in obj[k].keys():
                    keys.add(f"{k}.{k2}")
                    #print(f"{k}.{k2}")
            elif isinstance(obj[k], tuple) or isinstance(obj[k], list):
                for k2 in range(len(obj[k])):
                    keys.add(f"{k}.{k2}")
                    #print(f"{k}.{k2}")
            else:
                keys.add(k)
                #print(k)

    #print(keys)
        
    for key in keys:
        d[key] = []

    for obj in x["objects"]:
        donekeys = set()
        for k in keys:
            if k in obj.keys():
                d[k].append(obj[k])
                donekeys.add(k)
        for k in obj.keys():
            if isinstance(obj[k], dict):
                for k2 in obj[k].keys():
                    d[f"{k}.{k2}"].append(obj[k][k2])
                    donekeys.add(f"{k}.{k2}")
            elif isinstance(obj[k], tuple) or isinstance(obj[k], list):
                for k2 in range(len(obj[k])):
                    d[f"{k}.{k2}"].append(obj[k][k2])
                    donekeys.add(f"{k}.{k2}")
        #print(donekeys)
        for notdonekey in keys.difference(donekeys):
            d[notdonekey].append(None)


    #print(d)
    df = pd.DataFrame(data=d)

    #print(df.columns)

    #print(df.iloc[0])
    #print(df.iloc[1])

   


    rdf = df.loc[df["i"]==0]

    datadict = {}
    #print(rdf)
    datadict["changes"] = np.zeros((maxtime, maxlayer, MEMORYCOLUMNS))
    datadict["changes2"] = np.zeros((maxtime, MEMORYCOLUMNS))
    datadict["changes3"] = np.zeros((maxtime, MEMORYCOLUMNS))

    datadict["inputs"] = np.zeros((maxtime, INPUTMAXSIZE+1))
    datadict["outputs"] = np.zeros((maxtime, OUTPUTSIZE))
    datadict["total_o"] = np.zeros((maxtime, DOUTPUTSIZE))
    #output = np.zeros((maxtime, OUTPUTSIZE))

    datadict["read_vectors"] = np.zeros((maxtime, MEMORYCOLUMNS))

    def getDimTuple(key, keepalldim=False, start=1, getonedim=True):
        mtdim = [maxtime]
        otherdims = []
        for i in range(start, len(dims[key])):
            otherdims.append(dims[key][i])
        for d in otherdims:
            if keepalldim:
                mtdim.append(d)
            elif getonedim:
                mtdim.append(max(otherdims))
                break
            elif d > 1 and not getonedim:
                mtdim.append(d)
        
        if key == 'read_modes' or key == 'read_weights':
            mtdim[1] = mtdim[1] * read_heads
        return tuple(mtdim)

    datadict["link_matrix"] = np.zeros(getDimTuple('link_matrix', getonedim=False))
    datadict["precedence"] = np.zeros(getDimTuple('precedence'))


    
    
    for key in ["write_vector", 'read_keys', 'read_strengths', 'write_key', 'write_strength', 'erase_vector', 'free_gates', 'allocation_gate', 'read_modes', 'write_gate', 'write_weights', 'read_weights', 'content_weights', 'alloc', 'write_content_weights']:
        try:
            datadict[key] = np.zeros(getDimTuple(key))
        except:
            print(f"ERROR: {key}")
            

   
    def flattenspecial(arr):
        assert arr is not None
        assert isinstance(arr, np.ndarray)
        newshape = []
        npindexing = []
        for i in range(0, len(arr.shape)):
            if i!=1:
                newshape.append(arr.shape[i])
                npindexing.append(slice(None))
            else:
                npindexing.append(0)
        
        newarr = np.zeros(tuple(newshape))
        newarr = arr[tuple(npindexing)]
        return newarr
    
    #print(flattenspecial())


    for t in range(maxtime):
        for l in range(maxlayer):
            cdf = rdf.loc[rdf["time"]==t]
            cdf = cdf.loc[cdf["layer"]==l]

            

            for j in range(len(cdf)):
                ccdf = cdf.iloc[j]

                for key in ["precedence", "inputs", "outputs", "total_o", "read_vectors", "write_vector", "write_vector", 'read_keys', 'read_strengths', 'write_key', 'write_strength', 'erase_vector', 'free_gates', 'allocation_gate', 'read_modes', 'write_gate', 'write_weights', 'read_weights', 'content_weights', 'alloc', 'write_content_weights']:
                    if ccdf[key] is not None:
                        #print(key, ccdf[key].shape, datadict[key].shape)
                        flt = ccdf[key].flatten()
                        print(key)
                        datadict[key][t, 0:len(flt)] = np.round(flt, decimals=2)
                
                if ccdf["link_matrix"] is not None:
                    datadict["link_matrix"][t, :] = flattenspecial(ccdf["link_matrix"])

                if ccdf.loc["mem_before_w"] is None:
                    continue
                b_write = np.array(ccdf.loc["mem_before_w"])
                x_reset = np.array(ccdf.loc["mem_after_reset"])
                a_write = np.array(ccdf.loc["mem_after_w"])
                #print(b_write, a_write)
                datadict["changes"][t,l,:] = np.round(np.abs(a_write - b_write).flatten(),decimals=2)
                datadict["changes2"][t, :] = np.round(np.abs(a_write - b_write).flatten(),decimals=2)
                datadict["changes3"][t, :] = np.round(np.abs(x_reset - b_write).flatten(),decimals=2)
                

                #fig = px.density_heatmap(rdf, x="total_bill", y="tip")
                #fig.show()
    

    fig = px.imshow(datadict["changes2"], 
                    text_auto=True,
                    labels=dict(x="Memory Slot", y="Time", color="Change"),
                y=[str(t) for t in range(maxtime)],
                x=[f"slot_{j}" for j in range(MEMORYCOLUMNS)])
    fig.write_image(f"{name}/changes2.png", width=1920, height=1080)

    #print(datadict["changes2"])

    fig = px.imshow(datadict["changes3"], 
                    text_auto=True,
                    labels=dict(x="Memory Slot", y="Time", color="Change/Reset"),
                y=[str(t) for t in range(maxtime)],
                x=[f"slot_{j}" for j in range(MEMORYCOLUMNS)])
    fig.write_image(f"{name}/changes3.png", width=1920, height=1080)

    #print(datadict["changes3"])


    fig = px.imshow(np.round(datadict["inputs"], decimals=2), 
                    text_auto=True,
                    labels=dict(x="Input to NN", y="Time", color="Size"),
                y=[str(t) for t in range(maxtime)],
                x=[f"input_{j}" for j in range(INPUTMAXSIZE+1)])
    fig.write_image(f"{name}/inputs.png", width=1920, height=1080)

    #print(datadict["inputs"])

    fig = px.imshow(np.round(datadict["outputs"], decimals=2), 
                    text_auto=True,
                    labels=dict(x="hidden outputs of NN", y="Time", color="Size"),
                y=[str(t) for t in range(maxtime)],
                x=[f"output_{j}" for j in range(OUTPUTSIZE)])
    fig.write_image(f"{name}/outputs.png", width=1920, height=1080)

    #print(datadict["outputs"])

    fig = px.imshow(np.round(datadict["total_o"], decimals=2), 
                    text_auto=True,
                    labels=dict(x="outputs of NN", y="Time", color="Size"),
                y=[str(t) for t in range(maxtime)],
                x=[f"output_{j}" for j in range(DOUTPUTSIZE)])
    fig.write_image(f"{name}/total_o.png", width=1920, height=1080)

    #print(datadict["total_o"])

    fig = px.imshow(np.round(datadict["read_vectors"], decimals=2), 
                    text_auto=True,
                    labels=dict(x="Memory Read", y="Time", color="Value"),
                y=[str(t) for t in range(maxtime)],
                x=[f"slot_{j}" for j in range(MEMORYCOLUMNS)])
    fig.write_image(f"{name}/read_vectors.png", width=1920, height=1080)

    #print(datadict["read_vectors"])

    

    
    for key in ["write_vector", 'read_keys', 'read_strengths', 'write_key', 'write_strength', 'erase_vector', 'free_gates', 'allocation_gate', 'read_modes', 'write_gate', 'write_weights', 'read_weights', 'content_weights', 'alloc', 'write_content_weights']:
        try:
            xdim = max(list(getDimTuple(key)[1:]))
            #print(xdim, key, datadict[key].shape, datadict[key])
            fig = px.imshow(np.round(datadict[key], decimals=2), 
                            text_auto=True,
                            labels=dict(x=key, y="Time", color="Size"),
                        y=[str(t) for t in range(maxtime)],
                        x=[f"{key}_{j}" for j in range(xdim)])
            fig.write_image(f"{name}/{key}.png", width=1920, height=1080)
            #print(key)
            #print(datadict[key])
        except:
            print(f"ERROR: {key}")


    #print(datadict["precedence"].shape)
    fig = px.imshow(np.round(datadict["precedence"], decimals=2), 
                        text_auto=True,
                        labels=dict(x="precedence", y="Time", color="Size"),
                     y=[str(t) for t in range(maxtime)],
                     x=[f"precedence_{j}" for j in range(max(list(getDimTuple("precedence")[1:])))])
    fig.write_image(f"{name}/precedence.png", width=1920, height=1080)

    for t in range(maxtime):
        fig = go.Figure(data=go.Heatmap(
            z=datadict["link_matrix"][t],
            colorscale='Viridis'))
        fig.write_image(f"{name}/link_matrix_{t}.png", width=1920, height=1080)

        #print(datadict["link_matrix"])

        # fig= go.Figure(data=go.Heatmap(
        #     z=datadict["precedence"][t],
        #     colorscale='Viridis'))
        # fig.show()

    

   
    



Loading add_1f0/stepByStep.pkl
keys:  dict_keys(['stepByStep', 'CurrI', 'time', 'layer', 'currentObj', 'input', 'output', 'target', 'loss', 'MEMORYCOLUMNS', 'INPUTSIZE', 'OUTPUTSIZE', 'defObj', 'objects', 'read_heads', 'INTERMEDIATEOUTPUT', 'DNCOUPTPUT', 'accuracy'])
192 193
dict_keys(['stepByStep', 'CurrI', 'time', 'layer', 'currentObj', 'input', 'output', 'target', 'loss', 'MEMORYCOLUMNS', 'INPUTSIZE', 'OUTPUTSIZE', 'defObj', 'objects', 'read_heads', 'INTERMEDIATEOUTPUT', 'DNCOUPTPUT', 'accuracy'])
DIMLIST:  {'i': None, 'time': None, 'layer': None, 'inputs': (1, 27), 'outputs': (1, 192), 'o': None, 'total_o': (1, 193), 'chx': None, 'read_vectors': (1, 1), 'read_keys': (1, 1, 1), 'read_strengths': (1, 1), 'content_weights': (1, 1, 40), 'write_key': (1, 1, 1), 'write_strength': (1, 1), 'erase_vector': (1, 1, 1), 'write_vector': (1, 1, 1), 'free_gates': (1, 1), 'allocation_gate': (1, 1), 'read_modes': (1, 1, 3), 'write_gate': (1, 1), 'mem_after_w': (1, 40, 1), 'mem_before_w': (1, 40, 1)